# Advent of code 2020

This notebook contains my (somewhat documented) solutions to advent of code 2020. For
each day, I've tried to summarize ideas behind the solutions and to write understandable
and explicit code with type hints, so it's easier to follow what's happening.

If you want to tinker around with the solutions, you can install needed libraries
with either [poetry](https://python-poetry.org/) or just plain `pip install .`. You 
also need to setup the awesome [advent-of-code-data](
https://pypi.org/project/advent-of-code-data/) library, which I'm using for getting
input.

## Setup and shared functions

This first block is used to import all libraries, define all special function and
create all types needed in the notebook. These are mainly for parsing data into
different forms.

In [1]:
# Load black for automatic code formatting
%load_ext nb_black

# Load mypy for type checks. To type check, run %mypy in cell
%load_ext mypy_ipython

# Load libraries
import re
import math
import itertools
import functools
from abc import ABC, abstractmethod
from collections import defaultdict, deque, Counter
from typing import (
    Iterable,
    Callable,
    Optional,
    Literal,
    Iterator,
    Tuple,
    Set,
    List,
    Dict,
    Union,
    Deque,
)
from dataclasses import dataclass

import aocd
import networkx as nx
import numpy as np
import numba as nb
import scipy

Vector = Tuple[int, int]
Solution = Tuple[Union[int, str], Union[int, str]]


def data(day: int) -> str:
    """Get days input data as one string."""
    return aocd.get_data(day=day, year=2020)


def data_to_nums(day: int) -> List[int]:
    """Get days input data as numbers."""
    return [int(x) for x in re.findall(r"(\d+)", aocd.get_data(day=day, year=2020))]


def data_to_lines(day: int) -> List[str]:
    """Get days input data lines as list of strings."""
    return aocd.get_data(day=day, year=2020).splitlines()


def data_to_blocks(day: int) -> List[str]:
    """Get days input data blocks that are separated by empty new lines."""
    return [block for block in aocd.get_data(day=day, year=2020).split("\n\n")]


def test_day(day: int) -> None:
    """Run test for given day"""
    return globals()[f"test_day_{day}"]()


def run_day(day: int) -> Solution:
    """Return solutions for given day."""
    return globals()[f"day_{day}"]()


def print_day(day: int) -> None:
    """Print solutions for given day."""
    first, second = run_day(day=day)
    print("Part 1:", first)
    print("Part 2:", second)

<IPython.core.display.Javascript object>

## Day 1

**Task**: find the entries that sum to 2020.

Input size is quite small, just hundred numbers, so brute-forcing through every possible
combination is fast enough. Here, `combinations` from `itertools` generates possible
$k$-length combinations and when correct is found product of them is calculated with `math.prod`.

In [2]:
def solve_day_1(numbers: List[int], k: int) -> int:
    """Find first k-tuple of entries that has sum equivalent to goal."""
    goal = 2020
    entries = next(c for c in itertools.combinations(numbers, k) if sum(c) == goal)
    return math.prod(entries)

<IPython.core.display.Javascript object>

In [3]:
def test_day_1() -> None:
    example = [1721, 979, 366, 299, 675, 1456]
    assert solve_day_1(numbers=example, k=2) == 514579
    assert solve_day_1(numbers=example, k=3) == 241861950

<IPython.core.display.Javascript object>

In [4]:
def day_1() -> Solution:
    puzzle_input = data_to_nums(day=1)
    return (
        solve_day_1(numbers=puzzle_input, k=2),
        solve_day_1(numbers=puzzle_input, k=3),
    )

<IPython.core.display.Javascript object>

In [5]:
test_day(1)
print_day(1)

Part 1: 1010299
Part 2: 42140160


<IPython.core.display.Javascript object>

## Day 2

**Task**: find amount of passwords that match their counterpart rule.

Classic aocd-style input format! [Regex](https://en.wikipedia.org/wiki/Regular_expression) 
helps with parsing the input and [exclusive or (XOR)](https://en.wikipedia.org/wiki/Exclusive_or) helps at part two.

In [6]:
class PasswordValidatorBase(ABC):
    def __init__(self) -> None:
        """Initialize regex for splitting input."""
        self._splitter = re.compile(r"(\d+)-(\d+) ([a-z]): ([a-z]+)")

    def _parts(self, line: str) -> Tuple[int, int, str, str]:
        """Split input to parts and types."""
        match = self._splitter.match(line)
        if not match:
            raise ValueError(f"Couldn't get password and policy from line: {line}")
        low, high, char, string = match.groups()
        return int(low), int(high), char, string

    def count_valid_passwords(self, lines: List[str]) -> int:
        """Count valid passwords according to rules."""
        return sum(self._validate(*self._parts(line)) for line in lines)

    @abstractmethod
    def _validate(self, low: int, high: int, char: str, string: str) -> bool:
        """Validator method that should be implemented by subclass."""
        raise NotImplementedError


class PasswordValidator(PasswordValidatorBase):
    def _validate(self, low: int, high: int, char: str, string: str) -> bool:
        """Check if string has valid amount of occurences."""
        return low <= string.count(char) <= high


class ImprovedPasswordValidator(PasswordValidatorBase):
    def _validate(self, low: int, high: int, char: str, string: str) -> bool:
        """Check if char is at either, but not at both positions."""
        return (string[low - 1] == char) ^ (string[high - 1] == char)


password_validator = PasswordValidator()
improved_password_validator = ImprovedPasswordValidator()

<IPython.core.display.Javascript object>

In [7]:
def test_day_2() -> None:
    example = ["1-3 a: abcde", "1-3 b: cdefg", "2-9 c: ccccccccc"]
    assert password_validator.count_valid_passwords(example) == 2
    assert improved_password_validator.count_valid_passwords(example) == 1

<IPython.core.display.Javascript object>

In [8]:
def day_2() -> Solution:
    puzzle_input = data_to_lines(day=2)
    return (
        password_validator.count_valid_passwords(puzzle_input),
        improved_password_validator.count_valid_passwords(puzzle_input),
    )

<IPython.core.display.Javascript object>

In [9]:
test_day(2)
print_day(2)

Part 1: 524
Part 2: 485


<IPython.core.display.Javascript object>

## Day 3

**Task**: count the number of trees encountered while traversing the map.

Instead of really traversing the map, it's faster to generate possible 
points with `range` and `count` and calculate corresponding point in map 
with modulo. 

In [10]:
class TreeCounter:
    def __init__(self, area: List[str]) -> None:
        """Init area map, height and width."""
        self._area = area
        self._height = len(area)
        self._width = len(area[0])

    def _count_trees(self, slope: Vector) -> int:
        """Get sum of points where tree is encountered."""
        right, down = slope
        rows = range(0, self._height, down)
        cols = itertools.count(start=0, step=right)
        return sum(self._area[y][x % self._width] == "#" for y, x in zip(rows, cols))

    def _check_slopes(self) -> List[int]:
        """Get counts for all slopes in the puzzle."""
        slopes = ((3, 1), (1, 1), (5, 1), (7, 1), (1, 2))
        return [self._count_trees(slope=slope) for slope in slopes]

    def answers(self) -> Tuple[int, int]:
        """Get answer for both puzzle parts."""
        counts = self._check_slopes()
        return counts[0], math.prod(counts)

<IPython.core.display.Javascript object>

In [11]:
def test_day_3() -> None:
    example = [
        "..##.......",
        "#...#...#..",
        ".#....#..#.",
        "..#.#...#.#",
        ".#...##..#.",
        "..#.##.....",
        ".#.#.#....#",
        ".#........#",
        "#.##...#...",
        "#...##....#",
        ".#..#...#.#",
    ]
    counter = TreeCounter(area=example)
    assert counter.answers() == (7, 336)

<IPython.core.display.Javascript object>

In [12]:
def day_3() -> Solution:
    tree_counter = TreeCounter(area=data_to_lines(day=3))
    return tree_counter.answers()

<IPython.core.display.Javascript object>

In [13]:
test_day(3)
print_day(3)

Part 1: 280
Part 2: 4355551200


<IPython.core.display.Javascript object>

## Day 4

**Task**: count the number of valid passports

Since this puzzle is also about validation, I'm using the same format than in day 2 and
also a very useful data serializing library 
[marshmallow](https://marshmallow.readthedocs.io/en/stable/quickstart.html#required-fields).


In the first part, validator checks that group of expected fields is subset of 
fields in passport. Expected fields are all fields in schema except cid.
In the second part, all passports are validated against the schema.

In [14]:
from marshmallow import Schema, fields, validate, validates, ValidationError


class PassportSchema(Schema):
    """Marshmallow schema for validation."""

    byr = fields.Int(required=True, validate=validate.Range(min=1920, max=2002))
    iyr = fields.Int(required=True, validate=validate.Range(min=2010, max=2020))
    eyr = fields.Int(required=True, validate=validate.Range(min=2020, max=2030))
    hgt = fields.Str(required=True)
    hcl = fields.Str(required=True, validate=validate.Regexp(r"^#[0-9a-f]{6}$"))
    ecl = fields.Str(
        required=True,
        validate=validate.OneOf(["amb", "blu", "brn", "gry", "grn", "hzl", "oth"]),
    )
    pid = fields.Str(required=True, validate=validate.Regexp(r"^[0-9]{9}$"))
    cid = fields.Str()

    @validates("hgt")
    def _validate_hgt(cls, value: str) -> None:
        """Validate height field in passport"""
        try:
            unit = value[-2:]
            number = int(value[:-2])
        except (TypeError, ValueError):
            raise ValidationError("Invalid height")
        if unit == "cm" and 150 <= number <= 193:
            return
        if unit == "in" and 59 <= number <= 76:
            return
        raise ValidationError("Invalid height")


class PassportValidatorBase:
    def __init__(self) -> None:
        """Initialize regex for splitting input."""
        self._splitter = re.compile("(\w+):(\S+)")

    def parse_to_dict(self, line: str) -> Dict[str, str]:
        """Parse line to dictionary with fields and values."""
        return dict(self._splitter.findall(line))

    def count_valid_passports(self, lines: List[str]) -> int:
        """Count valid passports according to rules."""
        return sum(self._validate(self.parse_to_dict(line)) for line in lines)

    @abstractmethod
    def _validate(self, passport_dict: Dict[str, str]) -> bool:
        """Validator method that should be implemented by subclass."""
        raise NotImplementedError


class PassportValidator(PassportValidatorBase):
    def _validate(self, passport_dict: Dict[str, str]) -> bool:
        """Check if all needed fields are present."""
        schema_fields = set(PassportSchema().fields.keys())
        expected_fields = frozenset(schema_fields - {"cid"})
        return expected_fields.issubset(passport_dict.keys())


class ImprovedPassportValidator(PassportValidatorBase):
    def _validate(self, passport_dict: Dict[str, str]) -> bool:
        """Return false if schema validation has errors, else false."""
        return not bool(PassportSchema().validate(passport_dict))


passport_validator = PassportValidator()
improved_passport_validator = ImprovedPassportValidator()

<IPython.core.display.Javascript object>

In [15]:
def test_day_4() -> None:
    first_example = [
        "ecl:gry pid:860033327 eyr:2020 hcl:#fffffd byr:1937 iyr:2017 cid:147 hgt:183cm",
        "iyr:2013 ecl:amb cid:350 eyr:2023 pid:028048884 hcl:#cfa07d byr:1929",
        "hcl:#ae17e1 iyr:2013 eyr:2024 ecl:brn pid:760753108 byr:1931 hgt:179cm",
        "hcl:#cfa07d eyr:2025 pid:166559648 iyr:2011 ecl:brn hgt:59in",
    ]
    assert passport_validator.count_valid_passports(first_example) == 2

    second_example_invalid = [
        "eyr:1972 cid:100 hcl:#18171d ecl:amb hgt:170 pid:186cm iyr:2018 byr:1926",
        "iyr:2019 hcl:#602927 eyr:1967 hgt:170cm ecl:grn pid:012533040 byr:1946",
        "hcl:dab227 iyr:2012 ecl:brn hgt:182cm pid:021572410 eyr:2020 byr:1992 cid:277",
        "hgt:59cm ecl:zzz eyr:2038 hcl:74454a iyr:2023 pid:3556412378 byr:2007",
    ]
    second_example_valid = [
        "pid:087499704 hgt:74in ecl:grn iyr:2012 eyr:2030 byr:1980 hcl:#623a2f",
        "eyr:2029 ecl:blu cid:129 byr:1989 iyr:2014 pid:896056539 hcl:#a97842 hgt:165cm",
        "hcl:#888785 hgt:164cm byr:2001 iyr:2015 cid:88 pid:545766238 ecl:hzl eyr:2022",
        "iyr:2010 hgt:158cm hcl:#b6652a ecl:blu byr:1944 eyr:2021 pid:093154719",
    ]
    assert (
        improved_passport_validator.count_valid_passports(second_example_invalid) == 0
    )
    assert improved_passport_validator.count_valid_passports(
        second_example_valid
    ) == len(second_example_valid)

<IPython.core.display.Javascript object>

In [16]:
def day_4() -> Solution:
    puzzle_input = data_to_blocks(day=4)
    return (
        passport_validator.count_valid_passports(puzzle_input),
        improved_passport_validator.count_valid_passports(puzzle_input),
    )

<IPython.core.display.Javascript object>

In [17]:
test_day(4)
print_day(4)

Part 1: 192
Part 2: 101


<IPython.core.display.Javascript object>

## Day 5

**Task**: find out the highest seat ID and your own seat ID.

There's a nice catch in the puzzle: char pairs F/L and B/R aren't really different,
since each can be substituted with 0 or 1. The result is a [binary number
](https://en.wikipedia.org/wiki/Binary_number) string which can be then casted to int.

In the second part, potential seat ids are generated until correct one is found.

In [18]:
class SeatIdConverter:
    def __init__(self) -> None:
        """Initialize translation table from chars to nums."""
        self.table = str.maketrans("FBLR", "0101")

    def to_seat_id(self, line: str) -> int:
        """Convert binary space partioned line to seat id."""
        return int(line.translate(self.table), base=2)


id_converter = SeatIdConverter()

<IPython.core.display.Javascript object>

In [19]:
def test_day_5() -> None:
    assert id_converter.to_seat_id("BFFFBBFRRR") == 567
    assert id_converter.to_seat_id("FFFBBBFRRR") == 119
    assert id_converter.to_seat_id("BBFFBBFRLL") == 820

<IPython.core.display.Javascript object>

In [20]:
def day_5() -> Solution:
    seat_ids = {id_converter.to_seat_id(line) for line in data_to_lines(day=5)}
    my_seat_id = next(
        seat_id
        for seat_id in itertools.count(start=1)
        if seat_id not in seat_ids
        and seat_id - 1 in seat_ids
        and seat_id + 1 in seat_ids
    )

    return max(seat_ids), my_seat_id

<IPython.core.display.Javascript object>

In [21]:
test_day(5)
print_day(5)

Part 1: 858
Part 2: 557


<IPython.core.display.Javascript object>

## Day 6

**Task**: Count the number of questions to which anyone/everyone answered "yes".

In the first part, the question is basically "Which unique aphabets are present in any
persons answer in the group?". In the second part, question is "which unique alphabets are present on every 
persons answer in the group?", which can be handled with [set intersections
](https://en.wikipedia.org/wiki/Intersection_(set_theory)).

In [22]:
class AnswerCounter:
    @staticmethod
    def answered_by_anyone(group: str) -> Set[str]:
        """Return unique answers in group."""
        return {answer for answer in group if answer.isalpha()}

    def answered_by_everyone(self, group: str) -> Set[str]:
        """Return answers that we're persent in every persons answer."""
        return set.intersection(
            *(self.answered_by_anyone(person) for person in group.split("\n"))
        )

    def count(self, groups: List[str], attribute: Literal["anyone", "everyone"]) -> int:
        """Count amount of answers that satisfy given attribute (anyone/everyone)."""
        return sum(
            len(getattr(self, f"answered_by_{attribute}")(group)) for group in groups
        )


counter = AnswerCounter()

<IPython.core.display.Javascript object>

In [23]:
def test_day_6() -> None:
    example = ["abc", "a\nb\nc", "ab\nac", "a\na\na\na", "b"]
    assert counter.count(example, "anyone") == 11
    assert counter.count(example, "everyone") == 6

<IPython.core.display.Javascript object>

In [24]:
def day_6() -> Solution:
    puzzle_input = data_to_blocks(day=6)
    return (
        counter.count(puzzle_input, "anyone"),
        counter.count(puzzle_input, "everyone"),
    )

<IPython.core.display.Javascript object>

In [25]:
test_day(6)
print_day(6)

Part 1: 6532
Part 2: 3427


<IPython.core.display.Javascript object>

## Day 7

**Task**: Which bags can contain "shiny gold" bag / how many bags does "shiny gold" bag
contain?

A bit trickier one! One helpful observation is that the rules define a graph, a
[directed acyclic graph](https://en.wikipedia.org/wiki/Directed_acyclic_graph) (DAG) 
to be precise. So, the excellent network library [NetworkX](https://networkx.org/) can
be used.

Initially, graph contains weighted edges from container bag to content bags. The first 
part is solved by turning edges around to answer the question "which bags can contain
this bag". This way it's easy to walk graph through from the "shiny gold" node.

In second part, [depth-first-search](https://en.wikipedia.org/wiki/Depth-first_search)
is used to sum bag amounts from the innermost bags until "shiny gold" is reached.

In [26]:
WeightedEdge = Tuple[str, str, Dict[str, int]]


class BagCounter:
    def __init__(self, rules: List[str]) -> None:
        """Initialize graph."""
        self._graph = self._rules_to_graph(rules)

    def _rules_to_graph(self, rules: List[str]) -> nx.DiGraph:
        """Transform given list of rules into weighted graph."""
        splitter = re.compile(r"(\d+|\w+ \w+(?= bag))")

        def _edges(rule: str) -> Iterable[WeightedEdge]:
            """Get edges from rule by iterating content in pairs."""
            container, *content = splitter.findall(rule)
            if "no other" not in content:
                for amount, inner in zip(content[0::2], content[1::2]):
                    yield (str(container), str(inner), {"amount": int(amount)})

        return nx.DiGraph(
            itertools.chain.from_iterable(
                [edge for edge in _edges(rule)] for rule in rules
            )
        )

    def count_bags_containing(self) -> int:
        """Return amount of bags that contain shiny gold bag."""
        return len(nx.dag.descendants(self._graph.reverse(), "shiny gold"))

    def count_bags_inside(self) -> int:
        """Return amount of bags inside shiny gold bag."""

        def _count_containing(bag: str) -> int:
            return 1 + sum(
                self._graph[bag][inner]["amount"] * _count_containing(inner)
                for inner in self._graph.successors(bag)
            )

        return _count_containing("shiny gold") - 1

<IPython.core.display.Javascript object>

In [27]:
def test_day_7() -> None:
    example = [
        "light red bags contain 1 bright white bag, 2 muted yellow bags.",
        "dark orange bags contain 3 bright white bags, 4 muted yellow bags.",
        "bright white bags contain 1 shiny gold bag.",
        "muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.",
        "shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.",
        "dark olive bags contain 3 faded blue bags, 4 dotted black bags.",
        "vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.",
        "faded blue bags contain no other bags.",
        "dotted black bags contain no other bags.",
    ]
    counter = BagCounter(example)
    assert counter.count_bags_containing() == 4
    assert counter.count_bags_inside() == 32

    second_example = [
        "shiny gold bags contain 2 dark red bags.",
        "dark red bags contain 2 dark orange bags.",
        "dark orange bags contain 2 dark yellow bags.",
        "dark yellow bags contain 2 dark green bags.",
        "dark green bags contain 2 dark blue bags.",
        "dark blue bags contain 2 dark violet bags.",
        "dark violet bags contain no other bags.",
    ]
    second_counter = BagCounter(second_example)
    assert second_counter.count_bags_inside() == 126

<IPython.core.display.Javascript object>

In [28]:
def day_7() -> Solution:
    counter = BagCounter(data_to_lines(day=7))
    return counter.count_bags_containing(), counter.count_bags_inside()

<IPython.core.display.Javascript object>

In [29]:
test_day(7)
print_day(7)

Part 1: 372
Part 2: 8015


<IPython.core.display.Javascript object>

## Day 8

**Task**: Where the given program loops? / Change one instruction to make program terminate successfully.

This reminded me of [Intcode computer](https://adventofcode.com/2019/day/2) tasks from
AoC 2019, so I decided to just simulate the given program.

In the first part, loop breaks once some instruction is visited for the second time. In
the second part, generator function is used to generate potential tapes where either
"jmp" or "nop" is replaced until correct one is found.

In [30]:
Instruction = Tuple[str, int]
Tape = List[Instruction]


class Comp:
    def __init__(self, program: str) -> None:
        self._tape: Tape = [
            (op, int(arg)) for op, arg in re.findall(r"(\w+) ([+-]\d+)", program)
        ]

    def _run(
        self, tape: Tape, mode: Literal["find_loop", "find_terminating"]
    ) -> Optional[int]:
        """Run program from tape."""
        accumulator, head = 0, 0
        seen: Set[int] = set()
        while head not in seen:
            seen.add(head)
            try:
                op, arg = tape[head]
            except IndexError:
                return accumulator
            if op == "acc":
                accumulator += arg
            head += (movement := arg if op == "jmp" else 1)
        if mode == "find_loop":
            return accumulator
        return None

    def find_loop(self) -> int:
        """Run computer in loop finding mode"""
        result = self._run(self._tape, mode="find_loop")
        if not result:
            raise ValueError("Didn't find loop in program")
        return result

    def _potential_tapes(self) -> Iterator[Tape]:
        """Yield potential tape, where one 'jmp' or 'nop' is replaced."""
        replace = {"jmp": "nop", "nop": "jmp"}
        for i, (op, arg) in enumerate(self._tape):
            if op in replace:
                yield [*self._tape[:i], (replace[op], arg), *self._tape[i + 1 :]]

    def find_terminating(self) -> int:
        """Run computer with different tapes until terminating is found."""
        return next(
            result
            for tape in self._potential_tapes()
            if (result := self._run(tape, mode="find_terminating")) is not None
        )

<IPython.core.display.Javascript object>

In [31]:
def test_day_8() -> None:
    program = """nop +0
    acc +1
    jmp +4
    acc +3
    jmp -3
    acc -99
    acc +1
    jmp -4
    acc +6"""
    comp = Comp(program)
    assert comp.find_loop() == 5
    assert comp.find_terminating() == 8

<IPython.core.display.Javascript object>

In [32]:
def day_8() -> Solution:
    comp = Comp(program=data(day=8))
    return comp.find_loop(), comp.find_terminating()

<IPython.core.display.Javascript object>

In [33]:
test_day(8)
print_day(8)

Part 1: 2034
Part 2: 672


<IPython.core.display.Javascript object>

## Day 9

**Task**: What is the first number that isn't sum of any two numbers of the 25
numbers before it? / Find a contiguous subset of numbers that sum up to that first
number.

Time to brute-force again! In the first part, I had to check if number at
an index $i$ is a sum of any combination of previous 25 numbers. Number at first index
satisfying this property is the answer.

In the second part I decided to put a little of effort to produce an $O(n)$ solution 
(well, almost, calculating a sum isn't $O(1)$).
Solution uses a [sliding window](
https://www.geeksforgeeks.org/window-sliding-technique/) in order to have each
number added/removed from contiguous range max one time.

In [34]:
@dataclass
class EncryptionCracker:
    numbers: List[int]
    offset: int

    def _sum_in_interval(self, start: int, goal: int) -> bool:
        """Find out if number at goal is sum of any two numbers in interval."""
        return any(
            sum(combination) == self.numbers[goal]
            for combination in itertools.combinations(self.numbers[start:goal], 2)
        )

    def _find_invalid_number(self) -> int:
        """Find first number that is not sum of any n(= offset) numbers before it"""
        index = next(
            index
            for index in range(self.offset, len(self.numbers))
            if not self._sum_in_interval(start=index - self.offset, goal=index)
        )
        return self.numbers[index]

    def _find_weakness(self, invalid: int) -> Optional[int]:
        """Find a first contiguous set that sums up to invalid."""
        partial: Deque[int] = deque()
        for num in self.numbers:
            if sum(partial) == invalid and len(partial) >= 2:
                return min(partial) + max(partial)
            if sum(partial) < invalid:
                partial.append(num)
            while sum(partial) > invalid:
                partial.popleft()
        return None

    def crack(self) -> Tuple[int, int]:
        """Crack the XMAS-encryption!"""
        invalid = self._find_invalid_number()
        weakness = self._find_weakness(invalid)
        if not weakness:
            raise ValueError("Couldn't find weakness")
        return invalid, weakness

<IPython.core.display.Javascript object>

In [35]:
def test_day_9() -> None:
    example = [
        35,
        20,
        15,
        25,
        47,
        40,
        62,
        55,
        65,
        95,
        102,
        117,
        150,
        182,
        127,
        219,
        299,
        277,
        309,
        576,
    ]

    cracker = EncryptionCracker(numbers=example, offset=5)
    assert cracker.crack() == (127, 62)

<IPython.core.display.Javascript object>

In [36]:
def day_9() -> Solution:
    cracker = EncryptionCracker(numbers=data_to_nums(day=9), offset=25)
    return cracker.crack()

<IPython.core.display.Javascript object>

In [37]:
test_day(9)
print_day(9)

Part 1: 542529149
Part 2: 75678618


<IPython.core.display.Javascript object>

## Day 10

**Task**: Count differences between numbers / count total numbers of ways to arrange
numbers up to `n`th number.

It helps here to sort the given joltage array first. After that, both parts can be
solved in $O(n)$ by iterating through the sorted array. 

For the first part, differences between consecutive numbers are calculated. The 
second part can be solved with 
[dynamic programming](https://en.wikipedia.org/wiki/Dynamic_programming): since
one can add either adapter with $k+1$, $k+2$ or $k+3$ joltages after an adapter
with $k$ joltages, amount of adapters at some $k$ is based on amounts before it.
Answer is the amount of possible ways to sum up to charger with largest joltage.

In [38]:
def solve_day_10(numbers: List[int]) -> Solution:
    """Count differences between joltages and numbers of ways to arrange adapters."""
    joltages = sorted(numbers)

    ways: Dict[int, int] = defaultdict(int)
    diffs: Dict[int, int] = defaultdict(int)
    prev = 0
    ways[0] = 1
    for joltage in joltages:
        ways[joltage] = sum(ways[joltage - i] for i in [1, 2, 3])
        diffs[joltage - prev] += 1
        prev = joltage
    diffs[3] += 1

    return diffs[1] * diffs[3], ways[joltages[-1]]

<IPython.core.display.Javascript object>

In [39]:
def test_day_10() -> None:
    example = [16, 10, 15, 5, 1, 11, 7, 19, 6, 12, 4]
    assert solve_day_10(numbers=example) == (35, 8)
    second_example = [
        28,
        33,
        18,
        42,
        31,
        14,
        46,
        20,
        48,
        47,
        24,
        23,
        49,
        45,
        19,
        38,
        39,
        11,
        1,
        32,
        25,
        35,
        8,
        17,
        7,
        9,
        4,
        2,
        34,
        10,
        3,
    ]
    assert solve_day_10(numbers=second_example) == (220, 19208)

<IPython.core.display.Javascript object>

In [40]:
def day_10() -> Solution:
    return solve_day_10(numbers=data_to_nums(day=10))

<IPython.core.display.Javascript object>

In [41]:
test_day(10)
print_day(10)

Part 1: 1656
Part 2: 56693912375296


<IPython.core.display.Javascript object>

## Day 11

**Task**: Run [cellular automaton](https://en.wikipedia.org/wiki/Cellular_automaton)
until stable state is reached.

Game of life -style problems have been repeating puzzle theme in previous years, so it was nice to see one during 2020 too. Basic idea is quite simple: create new generations based on previous generation and a set of rules until consecutive generations are exactly the same.

For efficient solutions, following is needed: fast way to update whether seat is occupied or empty and fast way to get neighbours, especially on the second part. With pure python, fastest solution I could get was around 3 seconds, so I'll let this be the slowest solution this far :)

In [42]:
class SeatingSystem:
    def __init__(self, lines: List[str], mode: Literal["direct", "seen"]) -> None:
        """Initialize system with needed bookkeeping."""
        self._mode = mode
        self._limit = 5 if mode == "seen" else 4

        h, w = len(lines), len(lines[0])
        self._occupied: Dict[complex, bool] = {
            complex(x, y): False
            for y in range(h)
            for x in range(w)
            if lines[y][x] == "L"
        }
        self._floor: Set[complex] = {
            complex(x, y) for y in range(h) for x in range(w) if lines[y][x] == "."
        }
        self._neighbours = self._get_neighbours()

    def _get_neighbours(self) -> Dict[complex, List[complex]]:
        """Get list of neighbours for each point."""
        directions = [
            complex(x, y)
            for x, y in itertools.product((-1, 0, 1), repeat=2)
            if (x, y) != (0, 0)
        ]
        return {
            seat: [
                neighbour
                for direction in directions
                if (neighbour := self._get_neighbour(seat=seat, direction=direction))
                in self._occupied
            ]
            for seat in self._occupied.keys()
        }

    def _get_neighbour(self, seat: complex, direction: complex) -> Optional[complex]:
        """Get either direct neighbour (first part) or seen neighbours."""
        if self._mode == "seen":
            return next(
                point
                for multiplier in itertools.count(start=1)
                if (point := seat + multiplier * direction) not in self._floor
            )
        return seat + direction

    def _next_generation(self) -> Dict[complex, bool]:
        """Return next occupy statuses for each seat."""
        return {seat: self._becomes_occupied(seat) for seat in self._occupied}

    def _becomes_occupied(self, seat: complex) -> bool:
        """Find out if seat becomes occupied based on given rules."""
        count = sum(self._occupied[neighbour] for neighbour in self._neighbours[seat])
        currently_occupied = self._occupied[seat]
        if currently_occupied and count >= self._limit:
            return False
        if not currently_occupied and count == 0:
            return True
        return currently_occupied

    def run(self) -> int:
        """Run automaton until stable state has been reached."""
        while True:
            next_occupied = self._next_generation()
            if next_occupied == self._occupied:
                return sum(self._occupied.values())
            self._occupied = next_occupied

<IPython.core.display.Javascript object>

In [43]:
def test_day_11() -> None:
    example = [
        "L.LL.LL.LL",
        "LLLLLLL.LL",
        "L.L.L..L..",
        "LLLL.LL.LL",
        "L.LL.LL.LL",
        "L.LLLLL.LL",
        "..L.L.....",
        "LLLLLLLLLL",
        "L.LLLLLL.L",
        "L.LLLLL.LL",
    ]
    assert SeatingSystem(lines=example, mode="direct").run() == 37
    assert SeatingSystem(lines=example, mode="seen").run() == 26

<IPython.core.display.Javascript object>

In [44]:
def day_11() -> Solution:
    lines = data_to_lines(day=11)
    return (
        SeatingSystem(lines=lines, mode="direct").run(),
        SeatingSystem(lines=lines, mode="seen").run(),
    )

<IPython.core.display.Javascript object>

In [45]:
test_day(11)
print_day(11)

Part 1: 2281
Part 2: 2085


<IPython.core.display.Javascript object>

## Day 12

**Task**: Move ship by given rules, calculate distance between starting and ending
positions.

Quite straightforward puzzle: biggest difference between parts one and two is 
movement of direction (part 1) / waypoint (part 2). It also helps again to use
`complex` numbers for easier additions, multiplication and especially for doing
the rotations: code here uses the fact that raising complex number to the power 
of $n$ "moves" the number along perimeter of circle by multiplies of $90$ degrees. 
([Here's some info about the math behind](https://brilliant.org/wiki/complex-exponentiation/))

In [46]:
def solve_day_12(lines: List[str], mode: Literal["direct", "waypoint"]) -> int:
    """Run instructions from point (0,0) and return manhattan distance."""
    instructions = [(line[0], int(line[1:])) for line in lines]
    directions = {
        "N": complex(0, 1),
        "E": complex(1, 0),
        "S": complex(0, -1),
        "W": complex(-1, 0),
    }
    rotations = {"L": complex(0, 1), "R": complex(0, -1)}
    point = complex(0, 0)
    other = complex(10, 1) if mode == "waypoint" else directions["E"]

    for action, value in instructions:
        if action in rotations:
            other *= rotations[action] ** (value // 90)
        elif action == "F":
            point += other * value
        elif mode == "direct":
            point += directions[action] * value
        elif mode == "waypoint":
            other += directions[action] * value
    return int(abs(point.real) + abs(point.imag))

<IPython.core.display.Javascript object>

In [47]:
def test_day_12() -> None:
    example = ["F10", "N3", "F7", "R90", "F11"]

    assert solve_day_12(lines=example, mode="direct") == 25
    assert solve_day_12(lines=example, mode="waypoint") == 286

<IPython.core.display.Javascript object>

In [48]:
def day_12() -> Solution:
    lines = data_to_lines(day=12)
    return (
        solve_day_12(lines=lines, mode="direct"),
        solve_day_12(lines=lines, mode="waypoint"),
    )

<IPython.core.display.Javascript object>

In [49]:
test_day(12)
print_day(12)

Part 1: 1010
Part 2: 52742


<IPython.core.display.Javascript object>

## Day 13

**Task**: Given some timestamp, and a bus schedule, what is the earliest time to take
a bus (part 1) / the earliest time when busses depart exactly n minutes after first,
where n is their placement on the list.

For input, ids and diffs between bus id and its placement in schedule are calculated.
This helps on part 2.

The first part requires simple calculation, which gets remainder between the earliest
time and bus ids. The second part, on the other hand, requires some number
theory: [chinese remainder theorem](
https://en.wikipedia.org/wiki/Chinese_remainder_theorem). Idea here is that, there is
one integer $x$ that holds

$\begin{align}
x \equiv a_1 \pmod{n_1} \\
x \equiv a_2 \pmod{n_2} \\
x \equiv a_k \pmod{n_k} \\
\end{align}
$

where $a$ is a list of bus ids and $n$ is a list of differences between id and busses
placement in given schedule. Using the given example, this means that
`1068781 % 0 == (7-0) % 0`, `1068781 % 1 == (13-1) % 1`,`1068781 % 4 == (59-4) % 4` etc.

Sympy library [provides function](
https://docs.sympy.org/latest/modules/ntheory.html?highlight=baby%20step#sympy.ntheory.modular.crt)
to calculate the integer $x$.

In [50]:
from sympy.ntheory.modular import crt


def solve_day_13(notes: List[str]) -> Solution:
    """Solve the first possible departure time for both parts."""
    departure_time = int(notes[0])
    bus_ids, diffs = zip(
        *(
            (int(bus_id), int(bus_id) - i)
            for i, bus_id in enumerate(notes[1].split(","))
            if bus_id != "x"
        )
    )
    earliest_time, earliest_bus_id = min(
        (bus_id - departure_time % bus_id, bus_id) for bus_id in bus_ids
    )
    return earliest_time * earliest_bus_id, crt(bus_ids, diffs)[0]

<IPython.core.display.Javascript object>

In [51]:
def test_day_13() -> None:
    example = ["939", "7,13,x,x,59,x,31,19"]
    first_ans, second_ans = solve_day_13(example)
    assert first_ans == 295
    assert second_ans == 1068781

<IPython.core.display.Javascript object>

In [52]:
def day_13() -> Solution:
    return solve_day_13(data_to_lines(day=13))

<IPython.core.display.Javascript object>

In [53]:
test_day(13)
print_day(13)

Part 1: 3269
Part 2: 672754131923874


<IPython.core.display.Javascript object>

## Day 14

**Task**: Given a program with changing bitmask, memory addesses and values, what is 
the sum of the values in memory after executing the program?

[Mask article in Wikipedia](https://en.wikipedia.org/wiki/Mask_(computing)) is worth 
checking (it's actually linked in puzzle description as well), since the first part
is just applying two masks: other for masking bits to ones and other for masking bits
to zeros.

In the second part, things are bit more harder. I decided to go with following 
generator: apply ones mask and prepend address with zeros to make it 36-bit,
then generate all possible products of ones and zeros for "X"s in address and
yield results until all products have been consumed.

In [54]:
@dataclass
class Masks:
    generic: str
    zeros: int
    ones: int


class InitializationProgramBase:
    def __init__(self) -> None:
        """Initialize needed variables."""
        self._splitter = re.compile(r"(\d+)")
        self._memory: Dict[int, int] = {}
        self._masks = Masks(generic="", zeros=0, ones=0)

    def run(self, program: List[str]) -> int:
        """Execute masking from given program and return sum of memory values."""
        for line in program:
            operation, argument = line.split(" = ")
            if operation == "mask":
                self._masks.zeros = int(argument.replace("X", "1"), 2)
                self._masks.ones = int(argument.replace("X", "0"), 2)
                self._masks.generic = argument
            else:
                address, value = [int(x) for x in self._splitter.findall(line)]
                self._apply_masking(address, value)
        return sum(self._memory.values())

    @abstractmethod
    def _apply_masking(self, address: int, value: int) -> None:
        """Mask applier that should be implemented by subclass"""
        raise NotImplementedError


class SimpleInitializationProgram(InitializationProgramBase):
    def _apply_masking(self, address: int, value: int) -> None:
        """Apply both masks and write value to memory"""
        value &= self._masks.zeros
        value |= self._masks.ones
        self._memory[address] = value


class FloatingInitializationProgram(InitializationProgramBase):
    def _all_memory_addresses(self, address: int) -> Iterator[int]:
        """Generate all possible memory addresses."""
        address_36bit = f"{address | self._masks.ones:b}".zfill(36)
        mask = self._masks.generic
        for replacements in itertools.product("01", repeat=mask.count("X")):
            it = iter(replacements)
            result = [next(it) if a == "X" else b for a, b in zip(mask, address_36bit)]
            yield int("".join(result), 2)

    def _apply_masking(self, address: int, value: int) -> None:
        """Add value to each generated address."""
        for new_address in self._all_memory_addresses(address):
            self._memory[new_address] = value



<IPython.core.display.Javascript object>

In [55]:
def test_day_14() -> None:
    example = [
        "mask = XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X",
        "mem[8] = 11",
        "mem[7] = 101",
        "mem[8] = 0",
    ]
    assert SimpleInitializationProgram().run(program=example) == 165

    second_example = [
        "mask = 000000000000000000000000000000X1001X",
        "mem[42] = 100",
        "mask = 00000000000000000000000000000000X0XX",
        "mem[26] = 1",
    ]
    assert FloatingInitializationProgram().run(program=second_example) == 208

<IPython.core.display.Javascript object>

In [56]:
def day_14() -> Solution:
    puzzle_input = data_to_lines(day=14)
    return (
        SimpleInitializationProgram().run(program=puzzle_input),
        FloatingInitializationProgram().run(program=puzzle_input),
    )

<IPython.core.display.Javascript object>

In [57]:
test_day(14)
print_day(14)

Part 1: 17765746710228
Part 2: 4401465949086


<IPython.core.display.Javascript object>

## Day 15

**Task**: Play a game based on [Van Eck sequence](https://www.youtube.com/watch?v=etMJxB-igrc) and return $n$th number.

Game defined in the puzzle is quite simple: it's best to use data structure that allows fast lookups for last index where number was seen. 

The problem here is that Van Eck sequence is unpredictable and, as far as I know, there's no algorithm to solve $n$:th number of the sequence without calculating previous $n-1$ numbers.
I wasn't happy with basic cpython running time (around 7-8 seconds), but using numpy + numba gave some satisfying
results.

In [58]:
def solve_day_15(numbers: List[int]) -> Solution:
    @nb.njit
    def loop(seen: np.ndarray, turns: int, start: int, spoken: int) -> int:
        """Run loop for n=turns times, sped up with numba."""
        for turn in range(start, turns):
            last_seen = seen[spoken]
            seen[spoken] = turn
            spoken = last_seen if last_seen == 0 else turn - last_seen
        return spoken

    def solve(turns) -> int:
        seen = np.zeros(turns, dtype=np.int32)
        for i, num in enumerate(numbers):
            seen[num] = i + 1
        return loop(seen, turns, start=len(numbers), spoken=numbers[-1])

    return solve(turns=2020), solve(turns=30_000_000)

<IPython.core.display.Javascript object>

In [59]:
def test_day_15() -> None:
    assert solve_day_15(numbers=[0, 3, 6]) == (436, 175594)
    assert solve_day_15(numbers=[2, 1, 3]) == (10, 3544142)

<IPython.core.display.Javascript object>

In [60]:
def day_15() -> Solution:
    return solve_day_15(numbers=data_to_nums(day=15))

<IPython.core.display.Javascript object>

In [61]:
test_day(15)
print_day(15)

Part 1: 1618
Part 2: 548531


<IPython.core.display.Javascript object>

## Day 16

**Task**: Given a list of rules for ticket fields and some ticket examples, find out which rule corresponds to which field. Report error on invalid example tickets and values of some fields from your own ticket.

This one required quite much of parsing, but was quite straight-forwardly solvable after getting setup right. Parsing is done in following order: get ranges in rules and make set
of all allowed numbers (=numbers in ranges), then take all nearby tickets, check if ticket
has only valid values and if it doesn't, add values to error_rate. This is how the answer
to the first part is solved.

Second part requires some matching: each field corresponds to some rule and there is one
obvious way to do the matching. Therefor, [bipartite matching](https://en.wikipedia.org/wiki/Bipartite_graph) helps. One way to solve matching
would be to format problem as graph and find graph's max flow. However, scipy already has
good implementation, that takes a list of list of edges as an input.

In [62]:
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import maximum_bipartite_matching


@dataclass
class FieldRule:
    name: str
    ranges: List[int]
    position: Optional[int] = None


class TicketParser:
    def __init__(self, notes: List[str]) -> None:
        """Parse rules, my ticket and nearby tickets from original data."""
        self._error_rate = 0
        self._rules: List[FieldRule] = []
        self._valid_values: Set[int] = set()
        self._nearby_tickets: List[List[int]] = []

        raw_rules, raw_my_ticket, raw_nearby_tickets = notes
        self._parse_rules(raw_rules)
        self._parse_nearby_tickets(raw_nearby_tickets)
        self._match_field_nums_to_rules()
        self._my_ticket = self._parse_ticket(raw_ticket=raw_my_ticket.splitlines()[1])

    @staticmethod
    def _parse_ticket(raw_ticket: str) -> List[int]:
        """Parse values from single ticket"""
        return [int(x) for x in raw_ticket.split(",")]

    def _parse_rules(self, raw_rules: str) -> None:
        """Parse rule names and ranges from given rules and collect valid values to set."""
        splitter = re.compile(r"\w+(?=:)|\w+ \w+(?=:)|\d+")

        for raw_rule in raw_rules.splitlines():
            name, *ranges_raw = splitter.findall(raw_rule)
            ranges = [int(x) for x in ranges_raw]
            self._rules.append(FieldRule(name, ranges))
            self._valid_values |= set(
                (*range(ranges[0], ranges[1] + 1), *range(ranges[2], ranges[3] + 1))
            )

    def _parse_nearby_tickets(self, raw_nearby_tickets: str) -> None:
        """Parse nearby tickets and collect error rate from invalid tickets."""
        for raw_ticket in raw_nearby_tickets.splitlines()[1:]:
            ticket = self._parse_ticket(raw_ticket)
            if all(value in self._valid_values for value in ticket):
                self._nearby_tickets.append(ticket)
            else:
                self._error_rate += sum(
                    value for value in ticket if value not in self._valid_values
                )

    def _match_field_nums_to_rules(self) -> None:
        """Create mapping from field number to rule name."""

        def valid_for_rule(rule: FieldRule, field: int) -> bool:
            """Check if rule matches all fields at this position."""
            a, b, c, d = rule.ranges
            return all(
                a <= t[field] <= b or c <= t[field] <= d for t in self._nearby_tickets
            )

        potential_rules_per_position = [
            [valid_for_rule(rule, field=index) for rule in self._rules]
            for index, _ in enumerate(self._rules)
        ]
        positions = maximum_bipartite_matching(csr_matrix(potential_rules_per_position))
        for position, rule in zip(positions, self._rules):
            rule.position = position

    @property
    def error_rate(self) -> int:
        """Return answer to part one."""
        return self._error_rate

    def sum_of_departure_fields(self) -> int:
        """Return answer to part two."""
        names_per_position = {rule.position: rule.name for rule in self._rules}
        return math.prod(
            value
            for i, value in enumerate(self._my_ticket)
            if names_per_position[i].startswith("departure")
        )

<IPython.core.display.Javascript object>

In [63]:
def test_day_16() -> None:

    example = [
        "class: 1-3 or 5-7\nrow: 6-11 or 33-44\nseat: 13-40 or 45-50",
        "your ticket:\n7,1,14",
        "nearby tickets:\n7,3,47\n40,4,50\n55,2,20\n38,6,12",
    ]

    parser = TicketParser(notes=example)
    assert parser.error_rate == 71

    second_example = [
        "class: 0-1 or 4-19\nrow: 0-5 or 8-19\nseat: 0-13 or 16-19",
        "your ticket:\n11,12,13",
        "nearby tickets:\n3,9,18\n15,1,5\n5,14,9",
    ]
    second_parser = TicketParser(notes=second_example)
    # Test that class (first in list) has second pos, row has first and seat has third.
    assert second_parser._rules[0].position == 1
    assert second_parser._rules[1].position == 0
    assert second_parser._rules[2].position == 2

<IPython.core.display.Javascript object>

In [64]:
def day_16() -> Solution:
    puzzle_input = data_to_blocks(day=16)
    parser = TicketParser(notes=puzzle_input)
    return parser.error_rate, parser.sum_of_departure_fields()

<IPython.core.display.Javascript object>

In [65]:
test_day(16)
print_day(16)

Part 1: 20060
Part 2: 2843534243843


<IPython.core.display.Javascript object>

## Day 17

**Task**: Simulate [Conway's Game of life](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life), but in three and four dimensions.

Just like in day 11, this puzzle is a cellular automaton. Difference here is that a) grid is not limited, 
b) there are more than two dimensions and c) amount of round is given. 

Otherwise, same idea applies: get next generation based on given rules and make calculating neighbours efficient. 
Using sets as a way to represent points shines here, since looping through 4 dimension tables will be quite slow.

In [66]:
Cell = Tuple[int, ...]


class GameOfLife:
    def __init__(self, lines: List[str], dimensions: int) -> None:
        """Initialize cells, directions and amount of boot rounds."""
        self.cells = self.initial_cells(lines, dimensions)
        self.directions = [
            direction
            for direction in itertools.product((-1, 0, 1), repeat=dimensions)
            if direction != tuple(0 for _ in range(dimensions))
        ]
        self._boot_cycle = 6

    @staticmethod
    def initial_cells(lines: List[str], dimensions: int) -> Set[Cell]:
        """Get n-dimensional cells from given input."""
        h, w = len(lines), len(lines[0])
        return {
            (x, y, *(0 for _ in range(dimensions - 2)))
            for y in range(h)
            for x in range(w)
            if lines[y][x] == "#"
        }

    def run(self) -> int:
        """Run automaton for n=boot cycle times."""
        for _ in range(self._boot_cycle):
            self.cells = self.next_generation()
        return len(self.cells)

    def next_generation(self) -> Set[Cell]:
        """Get set of cells in next generation"""
        return {
            cell
            for cell, count in Counter(self.get_neighbours()).items()
            if self.becomes_active(cell=cell, count=count)
        }

    def becomes_active(self, cell: Cell, count: int) -> bool:
        """Check next status against game of life rules."""
        return count == 3 or (count == 2 and cell in self.cells)

    def get_neighbours(self) -> Iterator[Cell]:
        """Calculate all neighbours for cell"""
        return itertools.chain.from_iterable(
            [self.get_neighbour(cell, direction) for direction in self.directions]
            for cell in self.cells
        )

    @staticmethod
    def get_neighbour(cell: Cell, direction: Cell) -> Cell:
        """Return sum cell and direction, i.e. neighbour."""
        return tuple(a + b for a, b in zip(cell, direction))

<IPython.core.display.Javascript object>

In [67]:
def test_day_17() -> None:
    example = [".#.", "..#", "###"]
    assert GameOfLife(lines=example, dimensions=3).run() == 112
    assert GameOfLife(lines=example, dimensions=4).run() == 848

<IPython.core.display.Javascript object>

In [68]:
def day_17() -> Solution:
    puzzle_input = data_to_lines(day=17)
    return (
        GameOfLife(lines=puzzle_input, dimensions=3).run(),
        GameOfLife(lines=puzzle_input, dimensions=4).run(),
    )

<IPython.core.display.Javascript object>

In [69]:
test_day(17)
print_day(17)

Part 1: 368
Part 2: 2696


<IPython.core.display.Javascript object>

## Day 18

**Task**: Calculate bunch of expressions according to weird math rules. 

After some amount of googling if changing operator presedence would be possible in python (not easy or even possible), I decided to uses regexes to evaluate expressions in certain order. Method `run` searches for expressions inside parentheses, evaluates expressions (according to rules in part one / two) and replaces expression with evaluated answer. If there's parentheses inside parentheses, recursion goes a level deeper. 

Answer is found by just summing up everything together.

In [70]:
@dataclass
class Evaluator:
    mode: Literal["same presedence", "addition first"] = "same presedence"
    parentheses = re.compile(r"(\([^\(\)]*\))")
    add_or_mul = re.compile(r"\d+ [\+*] \d+")
    add = re.compile(r"\d+ [\+] \d+")
    mul = re.compile(r"\d+ [\*] \d+")

    def _run(self, line: str) -> int:
        """Evaluate single line recursively."""
        while expr := self.parentheses.search(line):
            inner_expr = expr[0][1:-1]
            line = self.parentheses.sub(str(self._run(inner_expr)), line, count=1)
        if self.mode == "same presedence":
            while expr := self.add_or_mul.search(line):
                line = self.add_or_mul.sub(str(eval(expr[0])), line, count=1)
        else:
            while expr := self.add.search(line):
                line = self.add.sub(str(eval(expr[0])), line, count=1)
            while expr := self.mul.search(line):
                line = self.mul.sub(str(eval(expr[0])), line, count=1)
        return int(line)

    def run_all(self, lines: List[str]) -> int:
        """Evaluate each line of the homework."""
        return sum(self._run(line) for line in lines)

<IPython.core.display.Javascript object>

In [71]:
def test_day_18() -> None:
    evaluator = Evaluator()
    assert evaluator._run("1 + 2 * 3 + 4 * 5 + 6") == 71
    assert evaluator._run("1 + (2 * 3) + (4 * (5 + 6))") == 51
    assert evaluator._run("2 * 3 + (4 * 5)") == 26
    assert evaluator._run("5 + (8 * 3 + 9 + 3 * 4 * 3)") == 437
    assert evaluator._run("5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))") == 12240
    assert evaluator._run("((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2") == 13632

    evaluator.mode = "addition first"
    assert evaluator._run("1 + (2 * 3) + (4 * (5 + 6))") == 51
    assert evaluator._run("2 * 3 + (4 * 5)") == 46
    assert evaluator._run("5 + (8 * 3 + 9 + 3 * 4 * 3)") == 1445
    assert evaluator._run("5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))") == 669060
    assert evaluator._run("((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2") == 23340

<IPython.core.display.Javascript object>

In [72]:
def day_18() -> Solution:
    puzzle_input = data_to_lines(day=18)
    return (
        Evaluator().run_all(lines=puzzle_input),
        Evaluator(mode="addition first").run_all(lines=puzzle_input),
    )

<IPython.core.display.Javascript object>

In [73]:
test_day(18)
print_day(18)

Part 1: 4297397455886
Part 2: 93000656194428


<IPython.core.display.Javascript object>

## Day 19

**Task**: From a list of recursively matching rules and list of messages, check what
messages match rule number 0.

Since given rules basically define a context-free grammar, my initial thought was to
use [Cocke–Younger–Kasami algorithm](https://en.wikipedia.org/wiki/CYK_algorithm) to
parse rules. After spending way too much time with that, I just decided to generate
regex string recursively. That turned out to be quite compact and good way to solve this
problem.

For the first part, `get_regex` method joins together regex strings recursively adding
logical OR:s where needed. The second parts changed rules are handled as follows:
`8: 42 | 42 8` means that rules number 42 can occur one or more times.
`11: 42 31 | 42 11 31` means that rules 42 and 31 must same amount of times, all 42:s
first and then all 31:s. I don't know if it's possible to create regex for this, so I
used kinda hacky solution instead: regex matches if there is either one 42 and one 31,
two 42:s and two 31:s etc. up to five times. Max five reps here comes just from
trial and error: it seemed to be the lowest amount to produce the correct amount.

In [74]:
class RegexFactor:
    def __init__(self, raw_rules: str) -> None:
        """Initialize rules, mode and regex."""
        self.mode: Literal["basic", "looping"] = "basic"
        self._rules: Dict[int, str] = {}
        for line in raw_rules.splitlines():
            rule_num, rule = line.split(": ")
            self._rules[int(rule_num)] = rule[1] if "a" in rule or "b" in rule else rule
        self._splitter = re.compile(r"\d+")

    def get_regex(self, rule_id: int) -> str:
        """Get regex for rule recursively."""
        if self.mode == "looping":
            if rule_id == 8:
                return self.get_regex(rule_id=42) + "+"
            if rule_id == 11:
                max_reps = 5
                return self.or_join(
                    self.get_regex(rule_id=42)
                    + f"{{{reps}}}"
                    + self.get_regex(rule_id=31)
                    + f"{{{reps}}}"
                    for reps in range(1, max_reps)
                )
        if self._rules[rule_id] in "ab":
            return self._rules[rule_id]
        return self.or_join(
            "".join(
                self.get_regex(rule_id=int(rule_id))
                for rule_id in self._splitter.findall(part)
            )
            for part in self._rules[rule_id].split(" | ")
        )

    @staticmethod
    def or_join(iterable: Iterable) -> str:
        """Join bunch of regex strings with logical OR."""
        return "(?:" + "|".join(iterable) + ")"


def solve_day_19(blocks: List[str]) -> Solution:
    """Return amount of matches against rule 0 in both modes."""

    def count_matches(messages: List[str], regex: str) -> int:
        """Count message matches against regex."""
        return sum(
            bool(re.fullmatch(regex, message)) for message in raw_messages.splitlines()
        )

    raw_rules, raw_messages = blocks

    factor = RegexFactor(raw_rules)
    basic_zero_regex = factor.get_regex(rule_id=0)
    factor.mode = "looping"
    looping_zero_regex = factor.get_regex(rule_id=0)
    messages = raw_messages.splitlines()

    return (
        count_matches(messages, regex=basic_zero_regex),
        count_matches(messages, regex=looping_zero_regex),
    )

<IPython.core.display.Javascript object>

In [75]:
def test_day_19() -> None:
    example = """0: 4 1 5
1: 2 3 | 3 2
2: 4 4 | 5 5
3: 4 5 | 5 4
4: "a"
5: "b"

ababbb
bababa
abbbab
aaabbb
aaaabbb""".split(
        "\n\n"
    )
    ans, _ = solve_day_19(blocks=example)
    assert ans == 2

    second_example = """42: 9 14 | 10 1
9: 14 27 | 1 26
10: 23 14 | 28 1
1: "a"
11: 42 31
5: 1 14 | 15 1
19: 14 1 | 14 14
12: 24 14 | 19 1
16: 15 1 | 14 14
31: 14 17 | 1 13
6: 14 14 | 1 14
2: 1 24 | 14 4
0: 8 11
13: 14 3 | 1 12
15: 1 | 14
17: 14 2 | 1 7
23: 25 1 | 22 14
28: 16 1
4: 1 1
20: 14 14 | 1 15
3: 5 14 | 16 1
27: 1 6 | 14 18
14: "b"
21: 14 1 | 1 14
25: 1 1 | 1 14
22: 14 14
8: 42
26: 14 22 | 1 20
18: 15 15
7: 14 5 | 1 21
24: 14 1

abbbbbabbbaaaababbaabbbbabababbbabbbbbbabaaaa
bbabbbbaabaabba
babbbbaabbbbbabbbbbbaabaaabaaa
aaabbbbbbaaaabaababaabababbabaaabbababababaaa
bbbbbbbaaaabbbbaaabbabaaa
bbbababbbbaaaaaaaabbababaaababaabab
ababaaaaaabaaab
ababaaaaabbbaba
baabbaaaabbaaaababbaababb
abbbbabbbbaaaababbbbbbaaaababb
aaaaabbaabaaaaababaa
aaaabbaaaabbaaa
aaaabbaabbaaaaaaabbbabbbaaabbaabaaa
babaaabbbaaabaababbaabababaaab
aabbbbbaabbbaaaaaabbbbbababaaaaabbaaabba""".split(
        "\n\n"
    )

    assert solve_day_19(blocks=second_example) == (3, 12)

<IPython.core.display.Javascript object>

In [76]:
def day_19() -> Solution:
    return solve_day_19(blocks=data_to_blocks(day=19))

<IPython.core.display.Javascript object>

In [77]:
test_day(19)
print_day(19)

Part 1: 195
Part 2: 309


<IPython.core.display.Javascript object>

## Day 20

**Task**: Reconstruct image from tiles, then count all the sea monsters in it.

This was a very time consuming puzzle and my original implementation was way, waaay more longer than this. After spending some time looking at Aoc reddit thread for better ideas, I decided to go with combination of numpy, scipy and networkX. Here, the basic idea is to first match each tile against others by their borders. Then, based on those borders, an initial grid with starting location for each border is made. Then, image is combined and sea monsters are calculated by applying scipy's correlate method: if some point has same amount of "#" -tiles in seamonster-size rectangle thatn seamonster has, then seamonster must be at that point. 

In [78]:
from scipy.ndimage import correlate


class ImageParser:
    def __init__(self, blocks: List[str]) -> None:
        """Initialize tiles."""
        self._get_num = re.compile("\d+")
        self._tiles = dict(self._parse_id_and_tile(block) for block in blocks)
        self._tile_size = len(next(iter(self._tiles.values())))
        self._form_image()

    def _parse_id_and_tile(self, block: str) -> Tuple[int, np.ndarray]:
        """Parse single id and tile from one block."""
        id_row, *tile_rows = block.splitlines()
        return int(self._get_num.findall(id_row)[0]), np.array(
            [[c == "#" for c in row] for row in tile_rows]
        )

    @staticmethod
    def _orientations(tile: np.ndarray) -> Iterator[np.ndarray]:
        """Generate all orientations for tiles."""
        for rotation in (np.rot90(tile, i) for i in range(1, 5)):
            yield rotation
            yield np.flipud(rotation)

    def _borders_match(self, tile_1: np.ndarray, tile_2: np.ndarray) -> bool:
        """Return matching tiles and direction their borders match."""
        for tile in self._orientations(tile_2):
            if (tile_1[0] == tile[-1]).all():
                return tile, (-1, 0)
            if (tile_1[-1] == tile[0]).all():
                return tile, (1, 0)
            if (tile_1[:, -1] == tile[:, 0]).all():
                return tile, (0, 1)
            if (tile_1[:, 0] == tile[:, -1]).all():
                return tile, (0, -1)

    def _create_border_graph(self) -> nx.Graph:
        """Return graph where each matching border has an edge from larger to smaller tile id."""
        return nx.Graph(
            [
                (int(id_1), int(id_2))
                for (id_1, tile_1), (id_2, tile_2) in itertools.product(
                    self._tiles.items(), repeat=2
                )
                if id_1 > id_2 and self._borders_match(tile_1, tile_2)
            ]
        )

    def _form_image(self) -> None:
        """Form the image from tiles by matching tiles one by one."""
        border_graph = self._create_border_graph()
        first_id = next(iter(self._tiles.keys()))
        stack = [(first_id, (0, 0))]
        locations = {(0, 0): first_id}
        seen: Set[int] = set()

        while stack:
            id_1, location = stack.pop()
            for id_2 in border_graph.neighbors(id_1):
                if id_2 in seen:
                    continue
                seen.add(id_2)
                tile, direction = self._borders_match(
                    self._tiles[id_1], self._tiles[id_2]
                )
                self._tiles[id_2] = tile
                next_location = tuple(a + b for a, b in zip(location, direction))
                stack.append((id_2, next_location))
                locations[next_location] = id_2

        n = int(math.sqrt(len(self._tiles)))
        m = self._tile_size - 2
        image = np.empty((n * m, n * m), dtype=np.int)
        offset_x = min(x for x, _ in locations)
        offset_y = min(y for _, y in locations)
        self._tile_positions = [[0] * n for _ in range(n)]

        for i in range(n):
            for j in range(n):
                tile_id = locations[i + offset_x, j + offset_y]
                tile = self._tiles[tile_id][1:-1, 1:-1]
                image[i * m : (i + 1) * m, j * m : (j + 1) * m] = tile
                self._tile_positions[i][j] = tile_id

        seamonster = [
            "                  # ",
            "#    ##    ##    ###",
            " #  #  #  #  #  #   ",
        ]
        kernel = np.array([c == "#" for line in seamonster for c in line]).reshape(
            (len(seamonster), len(seamonster[0]))
        )
        monster_size = kernel.sum()
        for orientation in self._orientations(image):
            matches = (
                correlate(orientation, kernel, mode="constant") == monster_size
            ).sum()
            if matches > 0:
                self._roughness = orientation.sum() - matches * monster_size

    @property
    def corner_id_prod(self):
        """Return product of corner ids."""
        n = int(math.sqrt(len(self._tiles)))
        corners = ((0, 0), (0, n - 1), (n - 1, 0), (n - 1, n - 1))
        return math.prod([self._tile_positions[i][j] for i, j in corners])

    @property
    def roughness(self):
        """Return roughness after it has been calculated."""
        return self._roughness

<IPython.core.display.Javascript object>

In [79]:
def test_day_20() -> None:
    example = """Tile 2311:
..##.#..#.
##..#.....
#...##..#.
####.#...#
##.##.###.
##...#.###
.#.#.#..##
..#....#..
###...#.#.
..###..###

Tile 1951:
#.##...##.
#.####...#
.....#..##
#...######
.##.#....#
.###.#####
###.##.##.
.###....#.
..#.#..#.#
#...##.#..

Tile 1171:
####...##.
#..##.#..#
##.#..#.#.
.###.####.
..###.####
.##....##.
.#...####.
#.##.####.
####..#...
.....##...

Tile 1427:
###.##.#..
.#..#.##..
.#.##.#..#
#.#.#.##.#
....#...##
...##..##.
...#.#####
.#.####.#.
..#..###.#
..##.#..#.

Tile 1489:
##.#.#....
..##...#..
.##..##...
..#...#...
#####...#.
#..#.#.#.#
...#.#.#..
##.#...##.
..##.##.##
###.##.#..

Tile 2473:
#....####.
#..#.##...
#.##..#...
######.#.#
.#...#.#.#
.#########
.###.#..#.
########.#
##...##.#.
..###.#.#.

Tile 2971:
..#.#....#
#...###...
#.#.###...
##.##..#..
.#####..##
.#..####.#
#..#.#..#.
..####.###
..#.#.###.
...#.#.#.#

Tile 2729:
...#.#.#.#
####.#....
..#.#.....
....#..#.#
.##..##.#.
.#.####...
####.#.#..
##.####...
##..#.##..
#.##...##.

Tile 3079:
#.#.#####.
.#..######
..#.......
######....
####.#..#.
.#...#.##.
#.#####.##
..#.###...
..#.......
..#.###...""".split(
        "\n\n"
    )
    parser = ImageParser(blocks=example)
    assert parser.corner_id_prod == 20899048083289
    assert parser.roughness == 273

<IPython.core.display.Javascript object>

In [80]:
def day_20() -> Solution:
    parser = ImageParser(blocks=data_to_blocks(day=20))
    return parser.corner_id_prod, parser.roughness

<IPython.core.display.Javascript object>

In [81]:
test_day(20)
print_day(20)

Part 1: 12519494280967
Part 2: 2442


<IPython.core.display.Javascript object>

## Day 21

**Task**: Parse allergens and match them with foods from encoded list. Report amount of foods without allergens and all foods with allergens.

This puzzle is actually almost like the one from the day 16, but instead of building bipartite graph, I decided to take another approach: first, find out all potentially allerginic foods by filtering them with set intersections. Then find out which allergen matches just one food, pop it out, remove food from all other allergen sets and continue until there's no allergens left.

Answer for first is solved by merging sets for dictionary `potential` and checking food appearances against those. Second is just sorting allergens by food.

In [82]:
Food = Allergen = str


def solve_day_21(lines: List[str]) -> Solution:
    potential_allergens: Dict[Allergen, Set[Food]] = defaultdict(set)
    food_appearances = Counter()
    final_allergens: List[Tuple[Food, Allergen]] = []
    regex = re.compile(r"(\w+)")

    for line in lines:
        foods, allergens = [regex.findall(part) for part in line.split("contains")]
        food_appearances.update(foods)
        for allergen in allergens:
            if allergen not in potential_allergens:
                potential_allergens[allergen] |= set(foods)
            potential_allergens[allergen] &= set(foods)

    allergenic_foods = functools.reduce(
        lambda x, y: {*x, *y}, potential_allergens.values()
    )

    while potential_allergens:
        allergen = next(
            allergen
            for allergen in potential_allergens
            if len(potential_allergens[allergen]) == 1
        )
        food = list(potential_allergens[allergen])[0]
        final_allergens.append((allergen, food))
        for foods in potential_allergens.values():
            foods.discard(food)
        del potential_allergens[allergen]

    first = sum(
        appearances
        for food, appearances in food_appearances.items()
        if food not in allergenic_foods
    )
    second = ",".join(food for _, food in sorted(final_allergens))

    return first, second

<IPython.core.display.Javascript object>

In [83]:
def test_day_21() -> None:
    example = [
        "mxmxvkd kfcds sqjhc nhms (contains dairy, fish)"
        "trh fvjkl sbzzf mxmxvkd (contains dairy)"
        "sqjhc fvjkl (contains soy)"
        "sqjhc mxmxvkd sbzzf (contains fish)"
    ]
    assert solve_day_21(example) == (5, "mxmxvkd,sqjhc,fvjkl")

<IPython.core.display.Javascript object>

In [84]:
def day_21() -> Solution:
    return solve_day_21(data_to_lines(day=21))

<IPython.core.display.Javascript object>

In [85]:
print_day(21)

Part 1: 2779
Part 2: lkv,lfcppl,jhsrjlj,jrhvk,zkls,qjltjd,xslr,rfpbpn


<IPython.core.display.Javascript object>

## Day 22

**Task**: Play games called "Combat" and "Recursive Combat".

Good old space cards from 2019, but this time, luckily, without a lot of arithmetics ([check this out if you want to see more](https://adventofcode.com/2019/day/22)). 

For efficiently simulating cards in decks, it's best to use deque: it supports $O(1)$ insertions and removals in both ends and also gives quite efficient way for slicing. This way
it's easy to run games according to given rules. 

In [86]:
Winner = str
Score = int


def solve_day_22(blocks: List[str]) -> Solution:
    def score(deck: Deque) -> Score:
        """Calculate score."""
        return sum((i + 1) * num for i, num in enumerate(reversed(deck)))

    def play_first(decks: List[Deque]) -> Deque:
        """Play basic version Combat."""
        while decks[0] and decks[1]:
            cards = [deck.popleft() for deck in decks]
            if cards[0] > cards[1]:
                decks[0].extend(cards)
            else:
                decks[1].extend(reversed(cards))
        return score(deck=decks[0]) if decks[0] else score(deck=decks[1])

    def play_second(decks: List[Deque], at_root=False) -> Union[Winner, Score]:
        """Play recursive version Combat."""
        seen = [set() for _ in range(2)]
        while decks[0] and decks[1]:
            if any(tuple(decks[i]) in seen[i] for i in range(2)):
                return "Player 1"
            seen[0].add(tuple(decks[0]))
            seen[1].add(tuple(decks[1]))
            cards = [deck.popleft() for deck in decks]
            if all(len(decks[i]) >= cards[i] for i in range(2)):
                new_decks = [
                    deque(itertools.islice(a, 0, b)) for a, b in zip(decks, cards)
                ]
                winner = play_second(decks=new_decks)
            else:
                winner = "Player 1" if cards[0] > cards[1] else "Player 2"
            if winner == "Player 1":
                decks[0].extend(cards)
            else:
                decks[1].extend(reversed(cards))
        winner = "Player 1" if decks[0] else "Player 2"
        if at_root:
            return score(deck=decks[0]) if decks[0] else score(deck=decks[1])
        return winner

    def run(mode: Literal["first", "second"]) -> Score:
        """Form decks and """
        decks = [deque([int(x) for x in nums.splitlines()[1:]]) for nums in blocks]
        return (
            play_second(decks=decks, at_root=True)
            if mode == "second"
            else play_first(decks=decks)
        )

    return run(mode="first"), run(mode="second")

<IPython.core.display.Javascript object>

In [87]:
def test_day_22() -> None:
    example = ["Player 1:\n9\n2\n6\n3\n1", "Player 2:\n5\n8\n4\n7\n10"]
    assert solve_day_22(example) == (306, 291)

<IPython.core.display.Javascript object>

In [88]:
def day_22() -> Solution:
    return solve_day_22(data_to_blocks(day=22))

<IPython.core.display.Javascript object>

In [89]:
test_day(22)
print_day(22)

Part 1: 35013
Part 2: 32806


<IPython.core.display.Javascript object>

## Day 23

**Task**: Play cups and balls magic trick for $n$ rounds.

This is one of those aoc-puzzles, where it's easy to write first version, then input size 
bumps up to something like $10^9$ and you're left with solution that runs for 2 hours.

Rules themselves are quite simple: take three elements and move them to another place without changing their order, then repeat. A good data structure for this is linked list, but since
python doesn't have one in standard library and only single linked list is needed (to know next cups on the right side), one can simulate linked list with regular list. Here, each index 
$i$ in the list `neighbour` tells which label is on the right side of cup label $i$.

And since simulating cup movements is basically just looping over same code $n$ times, it can
be sped up with numba.

In [90]:
def solve_day_23(labels: str) -> Solution:
    cups = [int(x) for x in labels]

    @nb.njit
    def run(curr: int, neighbour: np.ndarray, repeat: int) -> np.ndarray:
        """Run cup movements for n=repeat times."""
        n = len(neighbour) - 1
        for _ in range(repeat):
            first = neighbour[curr]
            second = neighbour[first]
            last = neighbour[second]
            values = {first, second, last}

            dest = n if curr == 1 else curr - 1
            while dest in values:
                dest = n if dest == 1 else dest - 1

            neighbour[curr] = neighbour[last]
            neighbour[last] = neighbour[dest]
            neighbour[dest] = first

            curr = neighbour[curr]

        return neighbour

    def solve(mode: Literal["first", "second"]) -> Union[int, str]:
        """Initialize neighbour lists and needed variables."""
        neighbour = np.empty(len(cups) + 1, dtype=np.int64)
        n = len(cups)
        for i in range(n - 1):
            neighbour[cups[i]] = cups[i + 1]
        curr = cups[0]

        if mode == "second":
            neighbour[cups[n - 1]] = n + 1
            neighbour = np.append(
                neighbour, np.arange(n + 2, 10 ** 6 + 2, dtype=np.int64)
            )
            neighbour[-1] = cups[0]
            repeat = 10 ** 7
        else:
            neighbour[cups[-1]] = cups[0]
            repeat = 100

        solution = run(curr=curr, neighbour=neighbour, repeat=repeat)

        if mode == "second":
            return solution[1] * solution[solution[1]]

        curr, ans = 1, []
        while (curr := solution[curr]) != 1:
            ans.append(str(curr))
        return "".join(ans)

    return solve(mode="first"), solve(mode="second")

<IPython.core.display.Javascript object>

In [91]:
def test_day_23() -> None:
    assert solve_day_23(labels="389125467") == ("67384529", 149245887792)

<IPython.core.display.Javascript object>

In [92]:
def day_23() -> Solution:
    return solve_day_23(labels=data(day=23))

<IPython.core.display.Javascript object>

In [93]:
test_day(23)
print_day(23)

Part 1: 58427369
Part 2: 111057672960


<IPython.core.display.Javascript object>

## Day 24

**Task**: Simulate cellular automaton, but this time with hexagonal grid.

Another cellular automaton -style puzzle, so I'll use the same class layout than in day 17. Biggest different here is of course the hexagonal layout. After reading some parts from [excellent tutorial](https://www.redblobgames.com/grids/hexagons/) at Red Blob Games blog, 
I decided to go with offset coordinate style. Otherwise, code below is almost the same as in day 17.

In [94]:
class Lobby:
    def __init__(self, lines: List[str]) -> None:
        """Initialize cells and directions."""
        self.directions = {
            "nw": complex(0, -1),
            "ne": complex(1, -1),
            "e": complex(1, 0),
            "se": complex(0, 1),
            "sw": complex(-1, 1),
            "w": complex(-1, 0),
        }
        self._cells = self.initial_cells(lines)

    def initial_cells(self, lines: List[str]) -> Set[complex]:
        """Get cells by running directions in given input."""
        regex = re.compile(r"e|se|sw|w|nw|ne")
        toggles = [
            sum(self.directions[direction] for direction in regex.findall(line))
            for line in lines
        ]
        return {
            point
            for point, toggle_amount in Counter(toggles).items()
            if toggle_amount % 2 == 1
        }

    def run(self, days: int) -> int:
        """Run automaton for n=days times."""
        for _ in range(days):
            self._cells = self.next_generation()
        return len(self._cells)

    def next_generation(self) -> Set[complex]:
        """Get set of cells in next generation"""
        return {
            cell
            for cell, count in Counter(self.get_neighbours()).items()
            if self.becomes_active(cell=cell, count=count)
        }

    def becomes_active(self, cell: complex, count: int) -> bool:
        """Check next status against the tile flipping rules."""
        return count == 2 or (count == 1 and cell in self._cells)

    def get_neighbours(self) -> Iterator[complex]:
        """Calculate all neighbours for cell"""
        return itertools.chain.from_iterable(
            [cell + direction for direction in self.directions.values()]
            for cell in self._cells
        )

    @property
    def cells(self) -> int:
        """Return cell amount."""
        return len(self._cells)

<IPython.core.display.Javascript object>

In [95]:
def test_day_24() -> None:
    example = [
        "sesenwnenenewseeswwswswwnenewsewsw",
        "neeenesenwnwwswnenewnwwsewnenwseswesw",
        "seswneswswsenwwnwse",
        "nwnwneseeswswnenewneswwnewseswneseene",
        "swweswneswnenwsewnwneneseenw",
        "eesenwseswswnenwswnwnwsewwnwsene",
        "sewnenenenesenwsewnenwwwse",
        "wenwwweseeeweswwwnwwe",
        "wsweesenenewnwwnwsenewsenwwsesesenwne",
        "neeswseenwwswnwswswnw",
        "nenwswwsewswnenenewsenwsenwnesesenew",
        "enewnwewneswsewnwswenweswnenwsenwsw",
        "sweneswneswneneenwnewenewwneswswnese",
        "swwesenesewenwneswnwwneseswwne",
        "enesenwswwswneneswsenwnewswseenwsese",
        "wnwnesenesenenwwnenwsewesewsesesew",
        "nenewswnwewswnenesenwnesewesw",
        "eneswnwswnwsenenwnwnwwseeswneewsenese",
        "neswnwewnwnwseenwseesewsenwsweewe",
        "wseweeenwnesenwwwswnew",
    ]
    lobby = Lobby(lines=example)
    assert lobby.cells == 10
    assert lobby.run(days=100) == 2208

<IPython.core.display.Javascript object>

In [96]:
def day_24() -> Solution:
    lobby = Lobby(lines=data_to_lines(day=24))
    return lobby.cells, lobby.run(days=100)

<IPython.core.display.Javascript object>

In [97]:
test_day(24)
print_day(24)

Part 1: 400
Part 2: 3768


<IPython.core.display.Javascript object>

## Day 25

**Task**: Given two public keys, find out the encryption key they share.

The final day, yeah! Exchange process detailed in the puzzle is a lot like [Diffie-
Hellman key exchange](https://en.wikipedia.org/wiki/Diffie%E2%80%93Hellman_key_exchange), 
and in this case, the secret integer (loop size) must be solved.

Transforming process is basically taking $k$th power of the subject number $7$
modulo $20201227$, where $k = $ loop size. Since public key $n$ is given, loop 
size can be found from the equation $7^k \equiv n \pmod{20201227}$. 

Instead of brute forcing loop sizes $1…k$ until correct is found, it's better to
calculate [discrete logarithm](https://en.wikipedia.org/wiki/Discrete_logarithm) of
$n$ to the base $7$ modulo $20201227$. Luckily, Sympy library provides
[suitable function for this](
https://docs.sympy.org/latest/modules/ntheory.html?#sympy.ntheory.residue_ntheory.discrete_log).

In [98]:
from sympy.ntheory import discrete_log


def solve_day_25(keys: List[int]) -> int:
    """Calculate encryption key based on given keys."""
    card_key, door_key = keys
    subject_number, modulo, loop_size = 7, 20201227, 0

    k = discrete_log(modulo, card_key, subject_number)
    return int(pow(door_key, k, modulo))

<IPython.core.display.Javascript object>

In [99]:
def test_day_25() -> None:
    assert solve_day_25(keys=[5764801, 17807724]) == 14897079

<IPython.core.display.Javascript object>

In [100]:
def day_25() -> Solution:
    return solve_day_25(keys=data_to_nums(day=25)), None

<IPython.core.display.Javascript object>

In [101]:
test_day(25)
print_day(25)

Part 1: 3286137
Part 2: None


<IPython.core.display.Javascript object>

# Timings
Most of the solutions should run under one seconds with a decent, modern laptop. Here's the report on 2018 15" Macbook Pro (2,8 Ghz i7):

In [102]:
import time


def timing():
    """Report on timing for all days."""
    print("Day  Secs.")
    print("===  =====")
    for day in range(1, 26):
        start = time.time()
        run_day(day)
        total = time.time() - start
        print(f"{day:2} {total:6.3f}")


%time timing()

Day  Secs.
===  =====
 1  0.148
 2  0.008
 3  0.001
 4  0.168
 5  0.002
 6  0.007
 7  0.023
 8  0.012
 9  0.015
10  0.001
11  2.761
12  0.002
13  0.001
14  0.390
15  1.066
16  0.018
17  0.404
18  0.095
19  0.016
20  1.818
21  0.002
22  1.543
23  3.690
24  0.434
25  0.001
CPU times: user 12.3 s, sys: 180 ms, total: 12.5 s
Wall time: 12.6 s


<IPython.core.display.Javascript object>